# The Spread of Covid Transmission in Indonesia for each Province

In [419]:
import requests
import json
import pandas as pd
import folium
import geopandas as gpd

## Import Dataset
from: [API Kawal Corona](https://kawalcorona.com/api/)

In [420]:
response = requests.get("https://api.kawalcorona.com/indonesia/provinsi/")
# response.json()
covid = pd.json_normalize(response.json()).drop(columns=['attributes.FID']).rename(columns={'attributes.Kode_Provi':'id','attributes.Provinsi':'provinsi','attributes.Kasus_Posi':'positif','attributes.Kasus_Semb':'sembuh','attributes.Kasus_Meni':'meninggal'}).sort_values(by='provinsi').reset_index(drop=True)

In [421]:
cityCoor = pd.read_csv('dataset/id.csv')
cityCoor = cityCoor[(cityCoor['capital'] == 'admin') | (cityCoor['capital'] == 'primary')][['lat','lng','admin_name']].rename(columns={'admin_name':'provinsi'}).sort_values('provinsi').reset_index(drop=True)[['provinsi','lat','lng']]
cityCoor['provinsi'].replace('Jakarta','DKI Jakarta',inplace=True)
cityCoor['provinsi'].replace('Kepulauan Bangka Belitung','Bangka Belitung',inplace=True)
cityCoor['provinsi'].replace('Yogyakarta','Daerah Istimewa Yogyakarta',inplace=True)

In [422]:
df = covid.merge(cityCoor)
df['id'].replace(91,92,inplace=True)
df['id'].replace(94,91,inplace=True)
df

,id,provinsi,positif,sembuh,meninggal,lat,lng
0,11,Aceh,10814,9367,432,5.5500,95.3175
1,51,Bali,44236,41608,1317,-8.6667,115.2167
2,19,Bangka Belitung,12822,11272,201,-2.1000,106.1000
3,36,Banten,47101,43892,1200,-6.1200,106.1503
4,17,Bengkulu,6630,5743,172,-3.7956,102.2592
5,31,DKI Jakarta,406205,393166,6625,-6.2146,106.8451
6,34,Daerah Istimewa Yogyakarta,38703,33636,948,-7.8014,110.3644
7,75,Gorontalo,5355,4990,161,0.5411,123.0594
8,15,Jambi,7407,5987,108,-1.5900,103.6100
9,32,Jawa Barat,277553,243650,3678,-6.9500,107.5667


In [423]:
indonesia = r'dataset/indonesia-prov.geojson'
indonesia_map = folium.Map([0.8, 118], zoom_start=5)
folium.Choropleth(geo_data=indonesia, 
                  data=df,
                  columns=['id','positif'],
                  key_on='feature.properties.kode',
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.5,
                  legend_name='Covid-19 Positive Case in Indonesia'
                 ).add_to(indonesia_map)

for i in range(len(df['lat'])):
    folium.Marker(df[['lat', 'lng']].loc[i].tolist(),
                  tooltip = df['provinsi'].loc[i],
                  popup = '<b>Positive:</b> {}\n<b>Cured</b>: {}\n<b>Deaths:</b> {}'.format(df['positif'][i],df['sembuh'][i],df['meninggal'][i]),
                  icon = folium.Icon(color='red', icon='info-sign')
                 ).add_to(indonesia_map)

In [424]:
indonesia_map

In [426]:
indonesia_map.save('The Spread of Covid Transmission in Indonesia for each Province.html')

## References
1. [From API to Pandas—Getting JSON Data With Python](https://python.plainenglish.io/from-api-to-pandas-getting-json-data-with-python-df127f699b6b)
1. [API Kawal Corona](https://kawalcorona.com/api/)
1. [Datasets - Data Riset dan Teknologi Covid-19 Indonesia](https://sinta.ristekbrin.go.id/covid/datasets)
1. [Kawal Covid 19 - Visualization](https://datawrapper.dwcdn.net/MwHOx/581/)
1. [Indonesia Cities Database - Simplemaps](https://simplemaps.com/data/id-cities)
1. [Indonesia Province Geojson Github Repository](https://github.com/ans-4175/peta-indonesia-geojson/blob/master/indonesia-prov.geojson)